In [1]:
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
# params
batch_size = 256
learning_rate = 1e-3
epochs = 10

In [3]:
# transform
trans = transforms.ToTensor()

In [4]:
# initialize the train and test datasets
train_loader = torch.utils.data.DataLoader(datasets.FashionMNIST(root='data', 
                                                                 train=True, 
                                                                 transform=trans, 
                                                                 download=True), 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.FashionMNIST(root='data', 
                                                                train=False, 
                                                                transform=trans, 
                                                                download=True), 
                                          batch_size=batch_size)

In [5]:
# network set
w1, b1 = torch.randn(200,784,requires_grad=True),\
        torch.zeros(200,requires_grad=True)
w2, b2 = torch.randn(200,200,requires_grad=True),\
        torch.zeros(200, requires_grad=True)
w3, b3 = torch.randn(10,200,requires_grad=True),\
        torch.zeros(10,requires_grad=True)

# init
torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)

tensor([[ 0.1372, -0.0885,  0.0172,  ...,  0.1417, -0.0181,  0.0783],
        [ 0.0340, -0.1043,  0.0035,  ..., -0.0964, -0.0462,  0.0486],
        [ 0.1324,  0.0727,  0.2301,  ..., -0.0340,  0.0664, -0.0265],
        ...,
        [-0.0102,  0.0315,  0.1673,  ..., -0.0608, -0.1819,  0.0796],
        [-0.0459,  0.1167,  0.0115,  ...,  0.0491, -0.0645,  0.0405],
        [-0.1204,  0.0028, -0.1266,  ...,  0.0795, -0.0147,  0.1155]],
       requires_grad=True)

In [6]:
# forward
def forward(x):
    x = x@w1.T + b1
    x = F.relu(x)
    
    x = x@w2.T + b2
    x = F.relu(x)
    
    x = x@w3.T + b3
    x = F.softmax(x)
    return x

In [8]:
# optimizer
optimizer = torch.optim.SGD([w1,w2,w3,b1,b2,b3], lr=learning_rate)

# loss function
loss_fn = nn.CrossEntropyLoss()

In [14]:
# training
for epoch in range(epochs):
    # forward
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.reshape(-1, 28*28)
        y_hat = forward(data)
        
        loss = loss_fn(y_hat, target)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx%100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
    
    # test
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data = data.reshape(-1,28*28)
        
        logits = forward(data)
        test_loss += loss_fn(logits, target).item()
        
        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()
    
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

C:\Users\26250\AppData\Local\Temp/ipykernel_14920/2866170835.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.289007
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.283295
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.277774

Test set: Average loss: 0.0091, Accuracy: 2270/10000 (23%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.273787
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.275539
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.257377

Test set: Average loss: 0.0090, Accuracy: 2623/10000 (26%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.245956
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.238339
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.235384

Test set: Average loss: 0.0090, Accuracy: 2850/10000 (28%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.241912
Train Epoch: 3 [25600/60000 (43%)]	Loss: 2.234566
Train Epoch: 3 [51200/60000 (85%)]	Loss: 2.219662

Test set: Average loss: 0.0089, Accuracy: 3091/10000 (31%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.227649
Train Epoch: 4 [25600/60000 (43%)]	Loss: 2.197665
Train Epoch: 4 [51200/60000 (85%)]	Loss: 2.211756

Test set: Average loss: 0.0088